# Darty web crawler

Réduction moyenne sur une page de résultats


In [1]:
import requests
import unittest
from bs4 import BeautifulSoup
from multiprocessing import Pool


In [2]:
website_prefixe = "https://www.darty.com/nav/recherche?s=prix_desc&npk=1&text="
website_suffixe = "&fa=756"


In [3]:
def _handle_request_result_and_build_soup(request_result):
    if request_result.status_code == 200:
        html_doc =  request_result.text
        soup = BeautifulSoup(html_doc,"html.parser")
        return soup


In [4]:
def _convert_string_to_float(string):
    temp = string.strip()
    if temp[0]== '(' : # small negative percentage
        return float(temp[1:-2])
    if temp[-1]== '-' : # -- NA value
        return 0.0
    temp = ''.join(temp.split(',')) # remove thousand's comma
    return float(temp)


In [5]:
def get_data(page):
    res = requests.get(website_prefixe + page + website_suffixe)
    soup = _handle_request_result_and_build_soup(res)
    l1 = soup.find_all("p", class_="darty_prix_barre_remise darty_small separator_top")
    return list(map(lambda x : int(x.text[2:-1]), l1))


In [6]:
pages = ["dell", "acer"]

In [7]:
p = Pool(6);
results = p.map(get_data, pages)

In [8]:
l = list(map(lambda l : sum(l) / float(len(l)), results))
list(zip(pages, l))

[('dell', 30.727272727272727), ('acer', 24.0)]